# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [57]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

aircraft_type = 'pax'  #pax or cargo

input_path = './Data/A320_inputs.xlsx'
input_sheet = 'A320'
# output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_A320.xlsx'

iterations = 1000
chunks = 1000  #'auto' or more than 500

In [58]:
from Tools import *
from Model import *

In [59]:
UP_dataframe = read_unit_processes(database_path) #reads Unit Processes from database
UP = unit_process_dataset(UP_dataframe) #creates Unit Processes xarray dataset

In [60]:
MP_data, EP_data = read_CF(database_path) #reads MP and EP factors from database
CF = CharactFactors(MP_data, EP_data, UP.Substances) #creates CharactFactors object
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data) #adjusting the NLT category CFs.
CF.MP.to_array();

# Analysis

In [61]:
inputs = read_inputs(input_path, input_sheet) #reads inputs
p = ParameterSet(inputs, iterations, chunks) #creates ParameterSet object
p = func_unit(aircraft_type, p) #creates funcional unit parameters (see Tools.py)

In [65]:
ff_ccd = p["ff_ccd"]
ff_reduct = 0.1
ff = ff_ccd * (1 - ff_reduct)
# ff.mean().compute()

p["ff_ccd"] = ff

In [66]:
OEW = p["OEW"]
weight_reduct = .2
weight = OEW * (1-weight_reduct)
# weight.mean().compute()

p["OEW"] = weight

In [67]:
inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p) #creates empty LCI object
inventory.run(); #runs LCI model (see Model.py)

In [68]:
aeco = LCIA.build(inventory, CF) #creating LCIA object

In [64]:
# CC_baseline = aeco.mean(by="sum").loc["CC"].compute().data
CC_baseline = 0.11253979

array(0.11253979)

In [69]:
CC_result = aeco.mean(by="sum").loc["CC"].compute().data

In [72]:
(CC_result - CC_baseline) / CC_baseline * 100 #change from baseline in %

8.714610545771102

In [56]:
while CC_result > CC_baseline:
    print(CC_result)
    CC_result = CC_result - 0.01

0.11263224028386216
0.10263224028386217
0.09263224028386217
0.08263224028386218
0.07263224028386218
0.06263224028386219
0.05263224028386219


# Export Results

In [53]:
aeco.save(output_path, LCI=False) #Export MP, EP, LCI and CTV result to NetCDF file.

In [22]:
#aeco.to_excel(output_path)